In [1]:
import requests
import json  
import pandas as pd
import numpy as np

In [2]:
response= requests.get('https://api.tfl.gov.uk/BikePoint/')
stations = response.json()

In [3]:
stations[0]

{'$type': 'Tfl.Api.Presentation.Entities.Place, Tfl.Api.Presentation.Entities',
 'id': 'BikePoints_1',
 'url': '/Place/BikePoints_1',
 'commonName': 'River Street , Clerkenwell',
 'placeType': 'BikePoint',
 'additionalProperties': [{'$type': 'Tfl.Api.Presentation.Entities.AdditionalProperties, Tfl.Api.Presentation.Entities',
   'category': 'Description',
   'key': 'TerminalName',
   'sourceSystemKey': 'BikePoints',
   'value': '001023',
   'modified': '2023-09-01T16:11:32.793Z'},
  {'$type': 'Tfl.Api.Presentation.Entities.AdditionalProperties, Tfl.Api.Presentation.Entities',
   'category': 'Description',
   'key': 'Installed',
   'sourceSystemKey': 'BikePoints',
   'value': 'true',
   'modified': '2023-09-01T16:11:32.793Z'},
  {'$type': 'Tfl.Api.Presentation.Entities.AdditionalProperties, Tfl.Api.Presentation.Entities',
   'category': 'Description',
   'key': 'Locked',
   'sourceSystemKey': 'BikePoints',
   'value': 'false',
   'modified': '2023-09-01T16:11:32.793Z'},
  {'$type': 'Tfl.

In [4]:
data =  pd.DataFrame()
s_names = []
s_lat = []
s_lon = []

NbBikes = []
NbEmptyDocks = []
NbDocks = []
NbStandardBikes = []
NbEBikes = []
s_id = []

In [5]:
len_lat = 0
for station in stations:
    s_names.append(station['commonName'])
    s_lat.append(float(station['lat']))
    s_lon.append(float(station['lon']))
    for add_property in station['additionalProperties']:
        if add_property['key'] == 'NbBikes':
            NbBikes.append(int(add_property['value']))
        if add_property['key'] == 'NbEmptyDocks':
            NbEmptyDocks.append(int(add_property['value']))     
        if add_property['key'] == 'NbDocks':
            NbDocks.append(int(add_property['value']))
        if add_property['key'] == 'NbStandardBikes':
            NbStandardBikes.append(int(add_property['value']))
        if add_property['key'] == 'NbEBikes':
            NbEBikes.append(int(add_property['value']))
        if add_property['key'] == 'TerminalName':
            s_id.append(int(add_property['value']))
        

In [6]:
len(s_lat)

796

In [7]:
data['Station_name'] = s_names
data['s_lat'] = s_lat
data['s_lon'] = s_lon
data['s_num_bikes'] = NbBikes
data['s_num_empty_docks'] = NbEmptyDocks
data['s_num_std_bikes'] = NbStandardBikes
data['s_num_e-bikes'] = NbEBikes
data['s_num_docks'] = data['s_num_bikes'] + data['s_num_empty_docks']
data['s_id'] = s_id

In [8]:
data

,Station_name,s_lat,s_lon,s_num_bikes,s_num_empty_docks,s_num_std_bikes,s_num_e-bikes,s_num_docks,s_id
0,"River Street , Clerkenwell",51.529163,-0.109970,16,1,16,0,17,1023
1,"Phillimore Gardens, Kensington",51.499606,-0.197574,13,23,12,1,36,1018
2,"Christopher Street, Liverpool Street",51.521283,-0.084605,9,23,9,0,32,1012
3,"St. Chad's Street, King's Cross",51.530059,-0.120973,13,10,13,0,23,1013
4,"Sedding Street, Sloane Square",51.493130,-0.156876,17,10,17,0,27,3420
...,...,...,...,...,...,...,...,...,...
791,"South Bermondsey Station, Bermondsey",51.488067,-0.057774,13,10,13,0,23,300062
792,"Crimscott Street, Bermondsey",51.495598,-0.078893,0,18,0,0,18,300061
793,"Brandon Street, Walworth",51.489102,-0.091548,17,4,16,1,21,300060
794,"The Blue, Bermondsey",51.492221,-0.062513,16,5,14,2,21,300059


In [9]:
# South East limits
se_limit_lat = 51.523811 # everthing lower than this value is south east
se_limit_lon = -0.101342 # everything greater than this value is south east

# East limits
e_limit_lat = 51.523811 # everything greater than this value is east
e_limit_lon = -0.022306 # everything greater than this is east

# South West limits
sw_limit_lat = 51.501631 # everthing lower than this value is south east
sw_limit_lon = -0.101342 # everything lower than this value is south east

# West limits
w_limit_lat = 51.501072  # everthing greater than this value is south east
w_limit_lon = -0.182406  # everthing lower than this value is south east

# Central limits
c_toplimit_lat = 51.501072 # everthing greater than this value is central
c_lowlimit_lat = 51.523811 # everything lower than this value is central

c_toplimit_lon = -0.101342 # everything lower than this value is central
c_lowlimit_lon = -0.182406 # everthing greater than this value is central

# North limits
n_toplimit_lon = -0.022306 # everything lower than this value is central
n_lowlimit_lon = -0.182406 # everthing greater than this value is central

n_lowlimit_lat = 51.523811 # everthing greater than this value is central

In [10]:
loc = []

In [11]:
for station in data.index:
    if data['s_lat'][station] > c_toplimit_lat and data['s_lat'][station]< c_lowlimit_lat and data['s_lon'][station]< c_toplimit_lon and data['s_lon'][station]>c_lowlimit_lon:
        loc.append('Central')
    elif data['s_lon'][station] < n_toplimit_lon and data['s_lon'][station] > n_lowlimit_lon and data['s_lat'][station] > n_lowlimit_lat:
        loc.append('North')
    elif data['s_lon'][station] < w_limit_lon and data['s_lat'][station] > w_limit_lat:
        loc.append('West')
    elif data['s_lon'][station] < sw_limit_lon and data['s_lat'][station] < sw_limit_lat:
        loc.append('South_West')  
    elif data['s_lon'][station] > e_limit_lon and data['s_lat'][station] > e_limit_lat:
        loc.append('East')     
    elif data['s_lon'][station] > se_limit_lon and data['s_lat'][station] < se_limit_lat:
        loc.append('East')    
    else:
        loc.append('Other')

In [12]:
data['Area_loc'] = loc

In [13]:
data

,Station_name,s_lat,s_lon,s_num_bikes,s_num_empty_docks,s_num_std_bikes,s_num_e-bikes,s_num_docks,s_id,Area_loc
0,"River Street , Clerkenwell",51.529163,-0.109970,16,1,16,0,17,1023,North
1,"Phillimore Gardens, Kensington",51.499606,-0.197574,13,23,12,1,36,1018,South_West
2,"Christopher Street, Liverpool Street",51.521283,-0.084605,9,23,9,0,32,1012,East
3,"St. Chad's Street, King's Cross",51.530059,-0.120973,13,10,13,0,23,1013,North
4,"Sedding Street, Sloane Square",51.493130,-0.156876,17,10,17,0,27,3420,South_West
...,...,...,...,...,...,...,...,...,...,...
791,"South Bermondsey Station, Bermondsey",51.488067,-0.057774,13,10,13,0,23,300062,East
792,"Crimscott Street, Bermondsey",51.495598,-0.078893,0,18,0,0,18,300061,East
793,"Brandon Street, Walworth",51.489102,-0.091548,17,4,16,1,21,300060,East
794,"The Blue, Bermondsey",51.492221,-0.062513,16,5,14,2,21,300059,East
